In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import math
import time
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score

import sys
sys.path.append('../src')
from preprocessing import *
from plotting import *
from utils import *

In [4]:
# Smote libraries
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Raw dataset

In [13]:
df_db_raw = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', 
                             '../datasets/raw/HT_Sensor_dataset.dat')
df_db_raw = reclassify_series_samples(df_db_raw)

In [19]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']

errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train_raw, df_test_raw = split_series_byID(0.8, df_db_raw)
    xtrain, ytrain = df_train_raw[features].values, df_train_raw['class'].values
    xtest, ytest = df_test_raw[features].values, df_test_raw['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=6, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=6, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())

Accuracy: 0.8572143593668403 +- 0.025515978212783497
f1-score: 0.8190647327201015 +- 0.03829195828833234
Accuracy (smote): 0.8458565971888294 +- 0.031579337552137786
f1-score (smote): 0.8221617082189414 +- 0.03645957483160785


# Raw dataset normalized

In [20]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']

errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train_raw, df_test_raw = split_series_byID(0.8, df_db_raw)
    df_train_raw, df_test_raw = norm_train_test(df_train_raw, df_test_raw, features)
    xtrain, ytrain = df_train_raw[features].values, df_train_raw['class'].values
    xtest, ytest = df_test_raw[features].values, df_test_raw['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=6, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=6, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())

Accuracy: 0.8201521580939907 +- 0.01508803526206297
f1-score: 0.7765723760844815 +- 0.023235452515250485
Accuracy (smote): 0.7955777021534507 +- 0.026829736189149724
f1-score (smote): 0.770908003822002 +- 0.028783269484552905


# Preprocessed dataset (not norm)

In [5]:
df_db_prep = group_datafiles_byID('../datasets/preprocessed/HT_Sensor_prep_metadata.dat', 
                             '../datasets/preprocessed/HT_Sensor_prep_dataset.dat')
df_db_prep = reclassify_series_samples(df_db_prep)

In [7]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']

errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train_prep, df_test_prep = split_series_byID(0.8, df_db_prep)
    xtrain, ytrain = df_train_prep[features].values, df_train_prep['class'].values
    xtest, ytest = df_test_prep[features].values, df_test_prep['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())

Accuracy: 0.8145749950763296 +- 0.03624886523203026
f1-score: 0.7634346612206693 +- 0.04680816205439765
Accuracy (smote): 0.8096434394719362 +- 0.04186775560946184
f1-score (smote): 0.7790379175601206 +- 0.04444332021126809


In [8]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']

errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train_prep, df_test_prep = split_series_byID(0.8, df_db_prep)
    df_train_prep, df_test_prep = norm_train_test(df_train_prep, df_test_prep, features)
    xtrain, ytrain = df_train_prep[features].values, df_train_prep['class'].values
    xtest, ytest = df_test_prep[features].values, df_test_prep['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())

Accuracy: 0.8340416407840581 +- 0.04791051558741142
f1-score: 0.7833658131015035 +- 0.060903767747132534
Accuracy (smote): 0.810705712532265 +- 0.05249736055141607
f1-score (smote): 0.7856029020354242 +- 0.056115858348222546


# Windows dataset (not norm)

In [4]:
with open('../datasets/preprocessed/window120_dataset.pkl', 'rb') as f: 
    df_db = pickle.load(f)

In [5]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity',
            'R1_mean', 'R2_mean', 'R3_mean', 'R4_mean', 'R5_mean', 'R6_mean', 'R7_mean',
            'R8_mean', 'Temp._mean', 'Humidity_mean', 'R1_std', 'R2_std', 'R3_std', 'R4_std',
            'R5_std', 'R6_std', 'R7_std', 'R8_std', 'Temp._std', 'Humidity_std']
errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train, df_test = split_series_byID(0.8, df_db)
    xtrain, ytrain = df_train[features].values, df_train['class'].values
    xtest, ytest = df_test[features].values, df_test['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())

Accuracy: 0.842697679663286 +- 0.03612150086276967
f1-score: 0.7997203574301185 +- 0.044495086486672615
Accuracy (smote): 0.8373088255373812 +- 0.03845074698631557
f1-score (smote): 0.821415832735829 +- 0.03494529439940311


# Windows dataset normalized

In [6]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity',
            'R1_mean', 'R2_mean', 'R3_mean', 'R4_mean', 'R5_mean', 'R6_mean', 'R7_mean',
            'R8_mean', 'Temp._mean', 'Humidity_mean', 'R1_std', 'R2_std', 'R3_std', 'R4_std',
            'R5_std', 'R6_std', 'R7_std', 'R8_std', 'Temp._std', 'Humidity_std']
errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train, df_test = split_series_byID(0.8, df_db)
    df_train, df_test = norm_train_test(df_train, df_test, features)
    xtrain, ytrain = df_train[features].values, df_train['class'].values
    xtest, ytest = df_test[features].values, df_test['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())

Accuracy: 0.8588394806584356 +- 0.029749994213194787
f1-score: 0.820860741656366 +- 0.040412007031441224
Accuracy (smote): 0.8568151494744827 +- 0.020655782451296067
f1-score (smote): 0.8451657203060917 +- 0.022927579949338098


In [9]:
df_db_raw = group_datafiles_byID('../datasets/raw/HT_Sensor_metadata.dat', 
                             '../datasets/raw/HT_Sensor_dataset.dat')
df_db_raw = reclassify_series_samples(df_db_raw)

features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']

errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train_raw, df_test_raw = split_series_byID(0.8, df_db_raw)
    xtrain, ytrain = df_train_raw[features].values, df_train_raw['class'].values
    xtest, ytest = df_test_raw[features].values, df_test_raw['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=6, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=6, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('==== Raw not norm ====')
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())


features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']

errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train_raw, df_test_raw = split_series_byID(0.8, df_db_raw)
    df_train_raw, df_test_raw = norm_train_test(df_train_raw, df_test_raw, features)
    xtrain, ytrain = df_train_raw[features].values, df_train_raw['class'].values
    xtest, ytest = df_test_raw[features].values, df_test_raw['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=6, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=6, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('==== Raw normalized ====')
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())



df_db_prep = group_datafiles_byID('../datasets/preprocessed/HT_Sensor_prep_metadata.dat', 
                             '../datasets/preprocessed/HT_Sensor_prep_dataset.dat')
df_db_prep = reclassify_series_samples(df_db_prep)


features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']

errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train_prep, df_test_prep = split_series_byID(0.8, df_db_prep)
    xtrain, ytrain = df_train_prep[features].values, df_train_prep['class'].values
    xtest, ytest = df_test_prep[features].values, df_test_prep['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('==== Prep not norm ====')
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())





features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']

errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train_prep, df_test_prep = split_series_byID(0.8, df_db_prep)
    df_train_prep, df_test_prep = norm_train_test(df_train_prep, df_test_prep, features)
    xtrain, ytrain = df_train_prep[features].values, df_train_prep['class'].values
    xtest, ytest = df_test_prep[features].values, df_test_prep['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('==== Prep normalized ====')
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())





with open('../datasets/preprocessed/window120_dataset.pkl', 'rb') as f: 
    df_db = pickle.load(f)


features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity',
            'R1_mean', 'R2_mean', 'R3_mean', 'R4_mean', 'R5_mean', 'R6_mean', 'R7_mean',
            'R8_mean', 'Temp._mean', 'Humidity_mean', 'R1_std', 'R2_std', 'R3_std', 'R4_std',
            'R5_std', 'R6_std', 'R7_std', 'R8_std', 'Temp._std', 'Humidity_std']
errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train, df_test = split_series_byID(0.8, df_db)
    xtrain, ytrain = df_train[features].values, df_train['class'].values
    xtest, ytest = df_test[features].values, df_test['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('==== Windows 120 not norm ====')
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())


features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity',
            'R1_mean', 'R2_mean', 'R3_mean', 'R4_mean', 'R5_mean', 'R6_mean', 'R7_mean',
            'R8_mean', 'Temp._mean', 'Humidity_mean', 'R1_std', 'R2_std', 'R3_std', 'R4_std',
            'R5_std', 'R6_std', 'R7_std', 'R8_std', 'Temp._std', 'Humidity_std']
errs_acc = []
errs_f1 = []
errs_acc_smote = []
errs_f1_smote = []

over_dict = {'banana': 175000, 'wine': 175000}
under_dict = {'background': 500000}

for i in range(7):
    df_train, df_test = split_series_byID(0.8, df_db)
    df_train, df_test = norm_train_test(df_train, df_test, features)
    xtrain, ytrain = df_train[features].values, df_train['class'].values
    xtest, ytest = df_test[features].values, df_test['class'].values

    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc.append(acc)
    errs_f1.append(f1)

    oversample = SMOTE(sampling_strategy=over_dict)
    undersample = RandomUnderSampler(sampling_strategy=under_dict)
    xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)
    xtrain, ytrain = undersample.fit_resample(xtrain, ytrain)   
    rfc = RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=7, n_jobs=-1)
    rfc.fit(xtrain, ytrain)
    y_pred = rfc.predict(xtest)
    acc = accuracy_score(ytest, y_pred)
    f1 = f1_score(ytest, y_pred, average='weighted')
    errs_acc_smote.append(acc)
    errs_f1_smote.append(f1)

errs_acc = np.asarray(errs_acc)
errs_f1 = np.asarray(errs_f1)
errs_acc_smote = np.asarray(errs_acc_smote)
errs_f1_smote = np.asarray(errs_f1_smote)
print('==== Windows 120 normalized ====')
print('Accuracy:', errs_acc.mean(), '+-', errs_acc.std())
print('f1-score:', errs_f1.mean(), '+-', errs_f1.std())
print('Accuracy (smote):', errs_acc_smote.mean(), '+-', errs_acc_smote.std())
print('f1-score (smote):', errs_f1_smote.mean(), '+-', errs_f1_smote.std())

==== Raw not norm ====
Accuracy: 0.8418960522995799 +- 0.03632088039392034
f1-score: 0.7971172515127904 +- 0.04712454772292015
Accuracy (smote): 0.8277699338947454 +- 0.03764657496658198
f1-score (smote): 0.8082176890316616 +- 0.041986899096288714
==== Raw normalized ====
Accuracy: 0.8435238958626406 +- 0.016988139870408495
f1-score: 0.8044838931123677 +- 0.02726273359176542
Accuracy (smote): 0.8352368865135366 +- 0.029350892504258912
f1-score (smote): 0.8167438646556081 +- 0.03198988701165813
==== Prep not norm ====
Accuracy: 0.8065367076851159 +- 0.029776930251357886
f1-score: 0.7572592024620618 +- 0.03996370131297966
Accuracy (smote): 0.7907933461773212 +- 0.029474943405134558
f1-score (smote): 0.7667767335193576 +- 0.03574504446815725
==== Prep normalized ====
Accuracy: 0.8518799497840609 +- 0.046531772282859415
f1-score: 0.8152035473309679 +- 0.06113266960072451
Accuracy (smote): 0.8308586053612501 +- 0.05510826568942438
f1-score (smote): 0.816378820351669 +- 0.057696944279277226
